### Ejercicio de K-Means
### Juan Pablo Fonseca 138263

Se pretende categorizar/clasificar un conjunto de datos usando el método k-means, así como entrenar un modelo de clasificación en cada cluster. 

La base de datos con la que se trabajará es "iris.data", que tiene información sobre las plantas iris.

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from math import fabs
from random import uniform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
df = pd.read_csv("iris.data",header=None, na_values = "?") # lectura de datos
numc=[] # lista con las clasificaciones pero en números

In [29]:
# paso a números la clasificación
for i in range(len(df)):
    curr = df[df.columns[-1]].iloc[i]
    if(curr == "Iris-setosa"):
        numc.append(1)
    elif(curr == 'Iris-versicolor'):
        numc.append(2)
    elif(curr == 'Iris-virginica'):
        numc.append(3)
numc = pd.Series(numc)
numc.astype('category')
df[4] = numc
df.head() # muestro el inicio de la base

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1


In [30]:
#separación de datos
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[0:-1]],df[df.columns[-1]], train_size=0.75)

In [31]:
# pasarlo a matrices
X_trainM = X_train.as_matrix()
Y_trainM = Y_train.as_matrix()
X_testM = X_test.as_matrix()
Y_testM = Y_test.as_matrix()

In [32]:
# acomodo los datos
sepalLength = X_trainM[:,0] # tenemos cuatro atributos sin contar la etiqueta (categoría)
sepalWidth = X_trainM[:,1]
petalLength = X_trainM[:,2]
petalWidth = X_trainM[:,3]
sepalLengthT = X_testM[:,0]
sepalWidthT = X_testM[:,1]
petalLengthT = X_testM[:,2]
petalWidthT = X_testM[:,3]
entrenamientoSeleccionados = np.column_stack((sepalLength,sepalWidth,petalLength,petalWidth))
pruebaSeleccionados = np.column_stack((sepalLengthT,sepalWidthT,petalLengthT,petalWidthT))


In [33]:
# Estandarizo
scaler = StandardScaler()
entrSel_escalados = scaler.fit_transform(entrenamientoSeleccionados)
pruebaSel_escalados = scaler.transform(pruebaSeleccionados)

#### Evaluación del modelo inicial

In [34]:
clf = DecisionTreeClassifier()
clf = clf.fit(X=entrSel_escalados, y=Y_trainM)
Y_test_prediction = clf.predict(pruebaSel_escalados)
conf = confusion_matrix(Y_testM, Y_test_prediction)
conf

array([[14,  0,  0],
       [ 0, 14,  0],
       [ 0,  1,  9]], dtype=int64)

In [35]:
print("Para la primera clase (precision y recall):")
total = float(conf[0][0]+ conf[0][1]+ conf[0][2]+ conf[1][0]+ conf[1][1]+ conf[1][2]+ conf[2][0]+ conf[2][1]+ conf[2][2])
#Accuracy (num buenos/total)
accuracy = float(conf[0][0] + conf[1][1] + conf[2][2]) / total
#Precision para la primera clase
precision = float(conf[0][0])/float(conf[0][0] + conf[0][1] + conf[0][2])
#Recall para la primera clase
recall = float(conf[0][0])/float(conf[0][0] + conf[1][0] + conf[2][0])
print("Accuracy: {0} Precision: {1} Recall: {2}".format(accuracy,precision,recall))

Para la primera clase (precision y recall):
Accuracy: 0.973684210526 Precision: 1.0 Recall: 1.0


Dado que la BD es realmente sencilla, el desempeño inicial es extraordinariamente bueno.

### K-Means

In [36]:
num_clusters = 5 # fue uno de los que mejor se comportó
modeloKM = KMeans(n_clusters=num_clusters)

In [37]:
etiquetasClusterEntr = modeloKM.fit_predict(entrSel_escalados)
entrSel_escalados = np.column_stack((entrSel_escalados,etiquetasClusterEntr)) # labels train
etiquetasClusterPru = modeloKM.predict(pruebaSel_escalados) # labels test
pruebaSel_escalados = np.column_stack((pruebaSel_escalados,etiquetasClusterPru))
entrSel_escalados = np.column_stack((entrSel_escalados,Y_trainM))
# para este punto las dos últimas columnas son etiquetas

In [38]:
# classif guardará a los clasificadores
classif = []
for i in range(num_clusters): # se recorren los clasificadores
    c = DecisionTreeClassifier() # se inicializa el clasificador
    classif.append(c) # dicho clasificador se agrega a la lista
    current_cluster = np.array([x for x in entrSel_escalados if x[-2] == i]) # se detecta el current cluster
    classif[i].fit(current_cluster[:,:-2], current_cluster[:,-1])
listaPredicciones = []
for x in pruebaSel_escalados:
    cluster_num = int(x[-1])
    sample = x[:-1].reshape(1, -1)
    listaPredicciones.append(classif[cluster_num].predict(sample)) # predicciones
listaPredicciones = np.array(listaPredicciones)

### Evaluación del segundo modelo

In [39]:
# matriz de confusión
conf = confusion_matrix(Y_testM, listaPredicciones)
conf

array([[14,  0,  0],
       [ 0, 12,  2],
       [ 0,  0, 10]], dtype=int64)

In [40]:
print("Para la primera clase (precision y recall):")
total = float(conf[0][0]+ conf[0][1]+ conf[0][2]+ conf[1][0]+ conf[1][1]+ conf[1][2]+ conf[2][0]+ conf[2][1]+ conf[2][2])
#Accuracy (num buenos/total)
accuracy = float(conf[0][0] + conf[1][1] + conf[2][2]) / total
#Precision para la primera clase
precision = float(conf[0][0])/float(conf[0][0] + conf[0][1] + conf[0][2])
#Recall para la primera clase
recall = float(conf[0][0])/float(conf[0][0] + conf[1][0] + conf[2][0])
print("Accuracy: {0} Precision: {1} Recall: {2}".format(accuracy,precision,recall))

Para la primera clase (precision y recall):
Accuracy: 0.947368421053 Precision: 1.0 Recall: 1.0


Se observa que el desempeño de hecho baja un poco, pero la diferencia es casi despreciable. Podemos concluir que ambos métodos son prácticamente igual de buenos.

Como trabajo futuro, habría que probar con distintas bases de datos para tener una mejor idea del desempeño del método.